In [ ]:
# Copyright 2019 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# CropNet – Detecção de doenças da mandioca

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/cropnet_cassava"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/hub/tutorials/cropnet_cassava.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/hub/tutorials/cropnet_cassava.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/hub/tutorials/cropnet_cassava.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
  <td>     <a href="https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelo do TF Hub</a>
</td>
</table>

Este notebook mostra como usar o modelo CropNet de [classificação de doenças da mandioca ](https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2) no **TensorFlow Hub**. O modelo classifica imagens de folhas de mandioca em uma de 6 classes: *bacterial blight, brown streak disease, green mite, mosaic disease, healthy ou unknown*.

Este Colab demonstra como:

- Carregar o modelo https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2 a partir do **TensorFlow Hub**
- Carregar o dataset [cassava](https://www.tensorflow.org/datasets/catalog/cassava) a partir do **TensorFlow Datasets (TFDS)**
- Classificar imagens de folhas de mandioca em 4 doenças distintas ou como saudável ou desconhecido
- Avaliar a *exatidão* do classificador e analisar a *robustez* do modelo quando aplicado em imagens fora do domínio.

## Importações e configuração

In [ ]:
!pip install matplotlib==3.2.2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [ ]:
#@title Helper function for displaying examples
def plot(examples, predictions=None):
  # Get the images, labels, and optionally predictions
  images = examples['image']
  labels = examples['label']
  batch_size = len(images)
  if predictions is None:
    predictions = batch_size * [None]

  # Configure the layout of the grid
  x = np.ceil(np.sqrt(batch_size))
  y = np.ceil(batch_size / x)
  fig = plt.figure(figsize=(x * 6, y * 7))

  for i, (image, label, prediction) in enumerate(zip(images, labels, predictions)):
    # Render the image
    ax = fig.add_subplot(x, y, i+1)
    ax.imshow(image, aspect='auto')
    ax.grid(False)
    ax.set_xticks([])
    ax.set_yticks([])

    # Display the label and optionally prediction
    x_label = 'Label: ' + name_map[class_names[label]]
    if prediction is not None:
      x_label = 'Prediction: ' + name_map[class_names[prediction]] + '\n' + x_label
      ax.xaxis.label.set_color('green' if label == prediction else 'red')
    ax.set_xlabel(x_label)

  plt.show()

## Dataset

Vamos carregar o dataset *cassava* a partir do TFDS.

In [ ]:
dataset, info = tfds.load('cassava', with_info=True)

Vamos conferir as informações do dataset para aprendermos mais sobre ele, como a descrição e citação, além de informações sobre quantos exemplos estão disponíveis.

In [ ]:
info

O dataset *cassava* tem imagens de folhas de mandioca, com 4 doenças distintas, além de folhas de mandioca saudáveis. O modelo pode prever todas essas classes, bem como uma sexta classe "desconhecido" quando o modelo não tem confiança quanto à previsão.

In [ ]:
# Extend the cassava dataset classes with 'unknown'
class_names = info.features['label'].names + ['unknown']

# Map the class names to human readable names
name_map = dict(
    cmd='Mosaic Disease',
    cbb='Bacterial Blight',
    cgm='Green Mite',
    cbsd='Brown Streak Disease',
    healthy='Healthy',
    unknown='Unknown')

print(len(class_names), 'classes:')
print(class_names)
print([name_map[name] for name in class_names])

Antes de alimentarmos o modelo com dados, precisamos fazer um pré-processamento. O modelo espera imagens 224x224 com valores de canais RGB no intervalo [0, 1]. Vamos normalizar e redimensionar as imagens.

In [ ]:
def preprocess_fn(data):
  image = data['image']

  # Normalize [0, 255] to [0, 1]
  image = tf.cast(image, tf.float32)
  image = image / 255.

  # Resize the images to 224 x 224
  image = tf.image.resize(image, (224, 224))

  data['image'] = image
  return data

Vamos dar uma olhada em alguns exemplos do dataset.

In [ ]:
batch = dataset['validation'].map(preprocess_fn).batch(25).as_numpy_iterator()
examples = next(batch)
plot(examples)

## Modelo

Vamos carregar o classificador a partir do TF Hub e obter algumas previsões. Além disso, vamos ver qual é a previsão do modelo para alguns exemplos.

In [ ]:
classifier = hub.KerasLayer('https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2')
probabilities = classifier(examples['image'])
predictions = tf.argmax(probabilities, axis=-1)

In [ ]:
plot(examples, predictions)

## Avaliação e robustez

Vamos mensurar a *exatidão* do nosso classificador em uma divisão do dataset. Também podemos avaliar a *robustez* do modelo avaliando seu desempenho para um dataset não relacionado a mandiocas. Ao usar imagens de outros datasets de plantas, como iNaturalist ou beans, o modelo deve quase sempre retornar *unknown*.

In [ ]:
#@title Parameters {run: "auto"}

DATASET = 'cassava'  #@param {type:"string"} ['cassava', 'beans', 'i_naturalist2017']
DATASET_SPLIT = 'test' #@param {type:"string"} ['train', 'test', 'validation']
BATCH_SIZE =  32 #@param {type:"integer"}
MAX_EXAMPLES = 1000 #@param {type:"integer"}


In [ ]:
def label_to_unknown_fn(data):
  data['label'] = 5  # Override label to unknown.
  return data

In [ ]:
# Preprocess the examples and map the image label to unknown for non-cassava datasets.
ds = tfds.load(DATASET, split=DATASET_SPLIT).map(preprocess_fn).take(MAX_EXAMPLES)
dataset_description = DATASET
if DATASET != 'cassava':
  ds = ds.map(label_to_unknown_fn)
  dataset_description += ' (labels mapped to unknown)'
ds = ds.batch(BATCH_SIZE)

# Calculate the accuracy of the model
metric = tf.keras.metrics.Accuracy()
for examples in ds:
  probabilities = classifier(examples['image'])
  predictions = tf.math.argmax(probabilities, axis=-1)
  labels = examples['label']
  metric.update_state(labels, predictions)

print('Accuracy on %s: %.2f' % (dataset_description, metric.result().numpy()))

## Saiba mais

- Saiba mais sobre o modelo no TensorFlow Hub: https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2
- Aprenda a criar um classificador de imagens personalizado executado em um celular com o [kit de aprendizado de máquina](https://developers.google.com/ml-kit/custom-models#tfhub) e a [versão deste modelo para TensorFlow Lite](https://tfhub.dev/google/lite-model/cropnet/classifier/cassava_disease_V1/1).